<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto </font></h1>



First thing to do in order to segment the neighnorhoods in torronto is to import the libraries needed 

In [2]:
import pandas as pd
import numpy as np
import requests
print('imported!')


imported!


In [3]:
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" 
wiki_page = requests.get(link)
wiki_data = pd.read_html(wiki_page.text) #reading the html page into a pandaframe 
wiki_data = wiki_data[0] #keeping only the first table we need 
df_read = wiki_data[wiki_data["Borough"] != "Not assigned"] #Dropping Borough which are not assigned
df_read = df_read.groupby(['Postal Code']).head() # Grouping by Postal Code
df_read= df_read.reset_index()
df_read.drop(['index'], axis = 'columns', inplace = True) # dropping the index column
df_read

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [4]:
print (" Answer to Q1 : the shape of the data is :")
df_read.shape # getting the shape of the data frame 

 Answer to Q1 : the shape of the data is :


(103, 3)

 **Question two**

So in order to get the geographical coordinates we use the link was given in the course 


In [5]:
url = 'http://cocl.us/Geospatial_data' # getting the cvs file from the url 

In [6]:
df_geo = pd.read_csv(url) # read the cvs into a dataframe 
df_geo.head() #sorting the first five rows of the data 

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Now we combine the two dataframes  

In [7]:
df_toronto =df_read.join(df_geo.set_index('Postal Code'), on='Postal Code')


In [8]:
df_toronto #sorting the data into screen

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [9]:
df_toronto.shape #getting the shape of the data frame 

(103, 5)

**Question three**

Now everything looks clear and good , now we can explore and cluster the neighborhoods in Toronto
Before we get the data and start exploring it, let's download all the dependencies that we will need.


In [10]:
from geopy.geocoders import Nominatim


In [18]:

address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The coordinates of Toronto are 43.6534817, -79.3839347.


In [19]:
import folium

In [21]:
# Creating the map of Toronto
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# adding markers to map
for latitude, longitude, borough, neighbourhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto